# Ranking Strategies Notebook
This notebook normalized cumulative, reliability, and decay metrics and ranks strategies highest to lowest based on these metrics.

## Import Libraries

In [1]:
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import  util
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from EquityHedging.analytics import hedge_metrics as hm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
    


## Import returns data

### Imports Daily, Weekly, Monthly, Quarterly and Yearly returns data

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**) - SPTR, M1WD, SX5T
* Decide if you want a Fixed Income Benchmark (**include_fi**) - True, False
* Select what equity hedging strategies to exclude (**strat_drop_list**). Below is the list of strategies:

    * 99%/90% Put Spread, Down Var, Vortex, VOLA , Dynamic Put Spread, VRR, GW Dispersion, Corr Hedge

In [2]:
equity_bmk = 'M1WD'
include_fi = False
weighted = [True, False]
strat_drop_list = ['99%/90% Put Spread', 'Vortex']
new_strat = True
returns = dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

##  Get Notional Weights 

In [3]:
#notional_weights = dm.get_notional_weights(returns['Monthly'])
notional_weights = [19.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.25, 1.0]
df_weights = get_df_weights(notional_weights, list(returns['Monthly'].columns), include_fi)
fmt.get_notional_styler(df_weights)

,M1WD,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var
Notional Weights (Billions),$19.00,$1.00,$1.00,$1.00,$1.00,$1.00,$0.25,$1.00
Percentage Weights,100.00%,5.26%,5.26%,5.26%,5.26%,5.26%,1.32%,5.26%
Strategy Weights,0.00%,16.00%,16.00%,16.00%,16.00%,16.00%,4.00%,16.00%


In [23]:
df = hm.get_hedge_metrics_to_normalize(returns, equity_bmk, notional_weights)
df

,Benefit,Down Reliability,Up Reliability,Convexity,Cost,Decay
Down Var,1.384568,-0.378368,0.245336,0.950600,-2.029530,205.0
VOLA,0.630722,-0.759925,-0.383986,1.246549,-1.661032,152.0
Dynamic Put Spread,0.892500,-0.649971,-0.602008,0.259934,-1.260329,123.0
VRR,2.316304,-0.066891,-0.009208,0.541243,-2.178475,85.0
GW Dispersion,1.040725,-0.474068,0.030755,0.345939,-1.032908,174.0
Corr Hedge,1.261181,-0.138395,0.124009,1.087390,-1.202728,303.0
Def Var,1.343204,-0.047311,-0.077938,0.223392,-0.465385,365.0
Weighted Hedges,0.642579,-0.680657,-0.121284,0.413746,-0.602808,248.0


In [28]:
#create new function in util with the following and call in get normalized data
#this function will convert the down reliability to positive
if 'Down Reliability' in df.columns:
    df_1 = df.copy()
    for x in df.index:
        df_1['Down Reliability'][x] = -(df['Down Reliability'][x])
df_1

,Benefit,Down Reliability,Up Reliability,Convexity,Cost,Decay
Down Var,1.384568,0.378368,0.245336,0.950600,-2.029530,205.0
VOLA,0.630722,0.759925,-0.383986,1.246549,-1.661032,152.0
Dynamic Put Spread,0.892500,0.649971,-0.602008,0.259934,-1.260329,123.0
VRR,2.316304,0.066891,-0.009208,0.541243,-2.178475,85.0
GW Dispersion,1.040725,0.474068,0.030755,0.345939,-1.032908,174.0
Corr Hedge,1.261181,0.138395,0.124009,1.087390,-1.202728,303.0
Def Var,1.343204,0.047311,-0.077938,0.223392,-0.465385,365.0
Weighted Hedges,0.642579,0.680657,-0.121284,0.413746,-0.602808,248.0


In [30]:
df_normal= summary.get_normalized_data(df)
df_normal


,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var,Weighted Hedges
Benefit,0.447231,0.000000,0.155304,1.000000,0.243241,0.374030,0.422692,0.007034
Down Reliability,0.535432,0.000000,0.154297,0.972523,0.401138,0.872183,1.000000,0.111236
Up Reliability,1.000000,0.257301,0.000000,0.699599,0.746761,0.856815,0.618485,0.567331
Convexity,0.710750,1.000000,0.035714,0.310656,0.119773,0.844444,0.000000,0.186045
Cost,0.086945,0.302052,0.535959,0.000000,0.668714,0.569583,1.000000,0.919781
Decay,0.428571,0.239286,0.135714,0.000000,0.317857,0.778571,1.000000,0.582143


In [44]:
n=len(df_normal.columns)
c=list(df_normal.columns)

import random
import numpy as np
r=random.sample(range(0,35),n)

symbols=pd.DataFrame(columns=c,index=[0])
symbols.iloc[0] = r
#TODO: make a list of symbols [0-32 and 46-52] and then call first n of list to assign symbols to strategy

In [45]:
met= list(df_normal.index)
n=len(df_normal.columns)
fig = go.Figure()
col=0
for col in df_normal.columns:
    fig.add_trace(go.Scatter(
        x=met,
        y=df_normal[col],
        marker=dict(size = 20),
        marker_symbol = symbols[col],
        mode= "markers",
        name=col
        ))
fig.update_layout(title="",
                xaxis_title="Hedge Metric",
                yaxis_title="Higher the Better")
fig.show()